# Introduction

Derivatives are used to solve a large variety of modern-day problems. There are three general methods used to calculate derivatives:
1. Symbolic differentiation 
2. Numerical differentiation
3. Automatic differentiation 

Symbolic differentiation can be very useful, but there are some functions that do not have a symbolic derivative. Additionally, symbolic differentiation can be very costly, as it may recalculate the same expressions many times, or the expression for the derivative may grow exponentially. Sometimes we can avoid these issues by numerically differentiating our function. Often this means using finite differences. The method of finite differences calculates derivative at point $x$ by using the following definition:

$$f'(x) = \lim_{h\to 0} f(x) \frac{f(x+h)-f(x)}{h}$$ 

Finite differences can also be very effective in certain situations. However, as with symbolic differentiation, finite differences has its problems. The biggest issue is that to obtain the most accurate estimate of $f'(x)$, we would like to make $h$ as small as possible; in fact, we would like $h$ to be infinitely small. However, we cannot *actually* make $h$ zero, and thus we must compromise and choose some small-but-not-zero value for $h$, which brings us to our second problem: we cannot precisely represent all numbers. Our machines only have a certain level of precision. When we compute our derivatives numerically we introduce error by approximating values to their closest machine equivalent. To avoid these issues, we turn to our third approach: automatic differentiation. 

# Background

Automatic differentiation (AD) allows us to calculate the derivative to machine precision while avoiding symbolic differentiation's shortcomings. Our package implements on version of AD, the forward mode, by using an extension of the real numbers called the "dual numbers." The forward mode of AD finds the derivative of all intermediate variables with respect to our independent variable and combines them into a final derivative using the chain rule.

AD can also be used in "reverse mode," which we will not discuss in detail her, but this method shares many of the same characteristics as forward mode. However, the reverse mode computes derivatives of the dependent variable with respect to the intermediate variables. 

#### Dual Numbers
To carry out the forward mode AD we utilize dual numbers. Dual numbers are defined as numbers of the form $x + x'\epsilon$, where $\epsilon^2=0$ and $x \in \mathbb{R}^n$. We use operator overloading to redefine elementary operations to suit our problem. To see why this is useful, let's examine how dual numbers behave under different mathematical operations:

Addition: $(x+x'\epsilon) + (y + y'\epsilon) = x+y + (x'+y')\epsilon$

Subtraction: $(x+x'\epsilon) - (y + y'\epsilon) = x-y + (x'-y')\epsilon$

So far, this is as we might expect.

Multiplication: $(x+x'\epsilon) \times (y + y'\epsilon) = xy + y(x')\epsilon+ x(y')\epsilon$

Our definition of $\epsilon$ allows the multiplication of dual numbers to behave like the product rule.

Division: $\frac{(x+x'\epsilon)}{(y + y'\epsilon)} = \frac{(x+x'\epsilon)(y - y'\epsilon)}{(y + y'\epsilon)(y - y'\epsilon)} = \frac{xy+xy'\epsilon-yx'\epsilon}{y^2} = \frac{x}{y}+\epsilon \frac{xy'-yx'}{y^2}$

Division also follows rules for derivatives.

Finally, observe how functions of dual numbers behave:

$f(x+x'\epsilon) = f(x)+\epsilon f'(x)x'$

Which implies the following:

$g(f(x+x'\epsilon)) = g(f(x)+\epsilon f'(x)x') = g(f(x))+\epsilon g'(f(x))f'(x)x'$

The above example illustrates how dual numbers can be used to simultaneously calculate the value of a function at a point, $g(f(x))$, and the derivative, $g'(f(x))f'(x)x'$.


#### Tracing the computational graph
By keeping track of the intermediate values of the derivative we can calculate the derivative of composition of many elementary functions. We can picture this decomposition as a graph or table. For example, consider the following function$^{1}$: $$f\left(x, y, z\right) = \dfrac{1}{xyz} + \sin\left(\dfrac{1}{x} + \dfrac{1}{y} + \dfrac{1}{z}\right).$$

If we want to evaluate $f$ at $\left(1, 2, 3\right)$, we can construct the following table which keeps track for the elementary function, current value, and the elementary function derivative (evaluated with respect to all our variables).

| Trace | Elementary Function | Current Value | Elementary Function Derivative | $\nabla_{x}$ Value  | $\nabla_{y}$ Value  | $\nabla_{z}$ Value  |
| :---: | :-----------------: | :-----------: | :----------------------------: | :-----------------: | :-----------------: | :-----------------: |
| $x_{1}$ | $x_{1}$ | $1$ | $\dot{x}_1$ | $1$ | $0$ | $0$ | 
| $x_{2}$ | $x_{2}$ | $2$ | $\dot{x}_2$ | $0$ | $1$ | $0$ | 
| $x_{3}$ | $x_{3}$ | $3$ | $\dot{x}_3$ | $0$ | $0$ | $1$ | 
| $x_{4}$ | $1/x_{1}$ | $1$ | $-\dot{x}_{1}/x_{1}^{2}$ | $-1$ | $0$ | $0$ | 
| $x_{5}$ | $1/x_{2}$ | $\frac{1}{2}$ | $-\dot{x}_{2}/x_{2}^{2}$ | $0$ | $-\frac{1}{4}$ | $0$ | 
| $x_{6}$ | $1/x_{3}$ | $\frac{1}{3}$ | $-\dot{x}_{3}/x_{3}^{2}$ | $0$ | $0$ | $-\frac{1}{9}$ | 
| $x_{7}$ | $x_4 x_5 x_6$ | $\frac{1}{6}$ | $x_4(x_5\dot{x}_6 + x_6\dot{x}_5) + x_5x_6\dot{x}_4$ | $-\frac{1}{6}$ | $-\frac{1}{12}$ | $-\frac{1}{18}$ | 
| $x_{8}$ | $x_4 + x_5 + x_6$ | $\frac{11}{6}$ | $\dot{x}_4 + \dot{x}_5 + \dot{x}_6$ | $-1$ | $-\frac{1}{4}$ | $-\frac{1}{9}$ | 
| $x_{9}$ | $sin(x_8)$ | $sin(\frac{11}{6})$ | $cos(x_8)\dot{x}_8$ | $-cos(\frac{11}{6})$ | $-\frac{1}{4}cos(\frac{11}{6})$ | $-\frac{1}{9}cos(\frac{11}{6})$ | 
| $x_{10}$ | $x_7 + x_9$ | $sin(\frac{11}{6})+\frac{1}{6}$ | $\dot{x}_7 + \dot{x}_9$ | $-cos(\frac{11}{6})-\frac{1}{6}$ | $-\frac{1}{4}cos(\frac{11}{6})-\frac{1}{12}$ | $-\frac{1}{9}cos(\frac{11}{6})-\frac{1}{18}$ | 

As this example shows, we can use AD for both scalar and vector functions. AD can also be used for vector valued functions. The follow sections will make the implementation of these varients clear.

$^1$Example from Harvard CS207 Homework 4

# Package Import Design

### Example package use

`>>> import AutoDiff as ad`

`>>> from AutoDiff import MathOps as ops`

Suppose the user wants to differentiate $5\sin(x)$

`>>> ad1 = ad.Const(5) `

`>>> ad2 = ops.sin(ad.Variable('x'))`

`>>> ad3 = ad1*ad2`

`>>> print(ad3.get_val({'x': np.pi/2}))`

` 5`

`>>> print(ad3.get_der('x', {'x': np.pi/2}))`

` 0`

Suppose instead the user would like the derivative of $e^{sin(x)}$

`>>> ad1 = ops.sin(Variable('x')) `

`>>> ad2 = ops.exp(ad1)`

`>>> print(ad2.get_val({'x': np.pi}))`

` 1`


Finally, what if the user wants to find the derivative of a multi-variable function? 
Let $f(x,y) = x \sin(xy)$

`>>> ad1 = ad.Variable("x")`

`>>> ad2 = ad.Variable("y")`

`>>> ad3 = ad1* ops.sin(ad1*ad2)`

`>>> print(ad3.get_val({'x': np.pi, 'y': 0}))`

`>>> 0`

`>>> print(ad3.get_dir('x', {'x': np.pi, 'y': 0})`

`>>> 0`

`>>> print(ad3.get_dir('y', {'x': np.pi, 'y': 0})`

`>>> pi*pi`

# Software Organization
### Directory Structure
```
AutoDiff/
    AutoDiff/
        AutoDiff.py
        tests.py
        README.md
        __init__.py
        setup.py
        LICENSE
```
### Modules and Functionality
We plan on including an AutoDiff module with classes DiffObj(), Variable(DiffObj), Const(DiffObj), MathOps(), and MathDiffObj(DiffObj). Their basic functionalities are as defined below in Implementation. Within these modules, we'll be using the math library to access functions like math.sin(), along with possibly numpy for handling large vectors.

### Testing
Our test suite will live within the AutoDiff directory, and we will be using both TravisCI and Coveralls to automate testing. 

### Distribution
We haven't decided on a distribution method yet, but we'll probably be using the recommended PyPI.

# Implementation

## Class Structure
We envisage having the following classes in our package:
1. Class DiffObj()
2. Class Variable(DiffObj)
3. Class Const(DiffObj)
4. Class MathOps(), with Inner Class MathDiffObj(DiffObj)

These are described along with their Class Attributes and Class Methods below.

### 1. Class DiffObj()
This will be a Superclass and every instance of this Super-class (or of any Sub-class which inherits from this), will basically expose access to its 'value' and its 'derivative'. A mathematical equivalent of a DiffObj object will be:
 * a constant such as $5.0$, which we will implement via a Sub-class 'Const'
 * a variable such as $x$, which we will implement via a Sub-class 'Variable'
 * a mathematical expression such as $x^2 + sin(y)$.
 
 **Class Attributes**:
 * name_list: A list of strings, this is contains names of all Variable objects which make this DiffObj. E.g. a DiffObj representing $x + xy + sin(z)$ will have its name_list as ['x','y','z'].
 
 **Class Methods**:
 * get_val:
         INPUT
         ======
         value_dict:    a dictionary, whose keys are string Variable names, and value is a float number.
         OUTPUT:
         ======
         result_val:    a float number, the result of evaluating the DiffObj at variable values given in value_dict. 
 * get_der:
         INPUT
         ======
         value_dict:    a dictionary, whose keys are string Variable names, and value is a float number. These are 
                        the values at which the derivative will be evaluated. This dictionary should provide values 
                        for all Variables which appear in the DiffObj.
         der_var_name:  optional, a string or list of strings, which represent names of variables with respect to 
                        which the derivative is required. If this argument is not provided, we return derivative with 
                        respect to all variables which appear in the DiffObj.
         OUTPUT:
         ======
         result_val:    a float number, the result of evaluating the DiffObj at variable values given in value_dict.
         result_der:    dictionary, whose keys are strings representing names of Variables with respect to which the
                        the derivative is required, and values are the derivative values.
                        
  **Overloaded Operators**: This will include common mathematical operators such as \_\_add\_\_, \_\_mul\_\_, \_\_sub\_\_, \_\_div\_\_, \_\_pow\_\_. These will allow addition, subtraction, multiplication, division, exponentiation between diff objects.

### 2. Class Variable(DiffObj)
This subclass inherits from DiffObj, and is basically used for representing a variable such as x or y.

**Class Attributes**:
* var_name: a string, such as 'x', which is unique to this Variable.

### 3. Class Constant(DiffObj)
This subclass inherits from DiffObj, and is basically used for representing a consant such as 7.0.

**Class Attributes**:
* const_val: a float number, such as 7.0.

### 4. Class MathOps()
This subclass is basically used for implementing differentiable functions, which include trigonometric functions such as $\sin{x}$, and others such as $e^x$ etc. It will return an instance of an Inner Class called MathDiffObj (which inherits from DiffObj). 

**Inner Class**:
* Class MathDiffObj (DiffObj): Inherits from DiffObj and implements get_val and get_der for the mathematical functions inside MathOps.

**Class Methods**:
* sin:
         INPUT
         ======
         input_diffobj: an instance of DiffObj, whose keys are string Variable names, and value is a float number.
         OUTPUT:
         ======
         math_obj:      an instance of Inner Class MathDiffObj, which inherits from DiffObj.
* Other differentiable functions such as cos(x), tan(x), log(x) etc. 

## Core Data Structures

There are two core data structures in our implementation:

1. **Lists**: The name_list (a list of strings) representing variable names, that is stored in every Diffobj instance to indicate the variables influencing that instance. Eg. for the DiffObj w, where w represents sin(x)+y, the name_list of Variable x is ['x'], the name_list of Variable y is ['y'] and the name_list of w is ['w','x','y'].

2. **Dictionaries**: The dictionary value_dict, an argument of DiffObj.get_der, containing names and values that indicate the point in space at which we need to compute derivative and evaluate an expression, for example in w.get_val(value_dict).

## External Dependencies

As of now we believe we will use the following third party libraries:
1. NumPy
2. Math

## Dealing with Elementary Functions
In our design, we will provide a Class called MathOps, whose class methods will be named after elementary functions such as $sin$. For example, the function $sin$ will accept a DiffObj instance as argument, and return an object of type MathDiffObj (which inherits from DiffObj, and implements get_val and get_der methods). MathDiffObj will be implemented as an Inner Class of MathOps because it will not be used outside of MathOps. The following cell shows the structure of these classes.

In [6]:
class DiffObj():
    def __init__(self):
        pass
    def get_val(self):
        raise NotImplementedError
    def get_der(self):
        raise NotImplementedError
        
class MathOps():
    def __init__(self):
        pass
    @classmethod
    def sin(cls, diff_obj):
        return MathDiffObj('sin', diff_obj)
    
    class MathDiffObj(DiffObj):
        def __init__(self, op_name, diff_obj):
            self.op_name = op_name
            self.dobj = diff_obj
        def get_val(self):
            if self.op_name == 'sin':
                return math.sin(self.dobj.get_val())
        def get_der(self):
            raise NotImplementedError